In [ ]:
import wandb
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score
from pathlib import Path
from torchvision import datasets, models
from torch.utils.data import DataLoader
from transformers import Adam, get_linear_schedule_with_warmup, AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

In [2]:
ROOT_DATASET_PATH = Path('./cifake/')
TRAIN_DATASET_PATH = ROOT_DATASET_PATH / 'train'
TEST_DATASET_PATH = ROOT_DATASET_PATH / 'test'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
checkpoint = "microsoft/resnet-152"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
image_processor

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [6]:
normalize = Normalize(mean=image_processor.image_mean,
                      std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
    [
        RandomResizedCrop(crop_size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(crop_size),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(
        image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


train_ds = datasets.ImageFolder(root=TRAIN_DATASET_PATH, transform=train_transforms)
test_ds = datasets.ImageFolder(root=TEST_DATASET_PATH, transform=val_transforms)

In [ ]:
model = models.resnet152(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model.to(DEVICE)

model

In [ ]:
BATCH_SIZE = 32
MODEL_NAME = checkpoint.split("/")[-1]
EPOCHS = 5
LEARNING_RATE = 5e-5

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"]
                               for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)
eval_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn)

criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=len(train_loader) * EPOCHS * 0.1,
    num_training_steps=len(train_loader) * EPOCHS
)

In [ ]:
wandb.login()
wandb.init(project="proyek-akhir-pcd", name="resnet152-5epoch-pytorch")

In [ ]:
def evaluate(model, loader):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for pixel_values, labels in loader:
            pixel_values, labels = pixel_values.to(DEVICE), labels.to(DEVICE)
            outputs = model(pixel_values)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')
    return total_loss / len(loader), f1

step = 0
for epoch in range(EPOCHS):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for pixel_values, labels in progress_bar:
        pixel_values, labels = pixel_values.to(DEVICE), labels.to(DEVICE)

        # Forward and backward
        optimizer.zero_grad()
        outputs = model(pixel_values)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Logging with WANDB
        wandb.log({'Train Loss': loss.item()}, step=step)

        # Every 200 steps, perform evaluation
        if step % 200 == 0:
            eval_loss, eval_f1 = evaluate(model, eval_loader)
            wandb.log({'Eval Loss': eval_loss, 'Eval F1': eval_f1}, step=step)

        # Every 1200 steps, save a checkpoint
        if step % 1200 == 0:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': loss,
            }, f"checkpoint_{step}.pth")

        progress_bar.set_postfix(train_loss=loss.item())
        step += 1

wandb.finish()